In [ ]:
from IPython.display import clear_output
!pip install timm
!pip install git+https://github.com/fbcotter/pytorch_wavelets
clear_output()

# Here's the deal, I don't want to use any melspectrograms, so I just gathered every bit of code I found cool together instead!
The list goes:
* MLP Variational Autoencoder (I think)     
* TabNet (Self attention)
* Efficientnet
* Wavelet calculation (in pytorch)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import torch.nn as nn
import torch.nn.functional as F
import torch

import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical
import math

from torchvision import models
import timm 
from pytorch_wavelets import DWT1DForward, DWT1DInverse,ScatLayer

from torch.utils.data import Dataset,DataLoader

import gc
import time
import matplotlib.pyplot as plt

# This is an implementation of an MLP variational autoencoder

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        all_feat_cols = np.arange(143360)
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

#         self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

#         x = self.dense5(x)

        return x

# Everything Related to Tabnet is below

In [ ]:
class Sparsemax(nn.Module):
    def __init__(self, dim=None):
        super(Sparsemax, self).__init__()
        self.dim = -1 if dim is None else dim

    def forward(self, input):
        input = input.transpose(0, self.dim)
        original_size = input.size()
        input = input.reshape(input.size(0), -1)
        input = input.transpose(0, 1)
        dim = 1

        number_of_logits = input.size(dim)
        
        input = input - torch.max(input, dim=dim, keepdim=True)[0].expand_as(input)
        zs = torch.sort(input=input, dim=dim, descending=True)[0]
        range = torch.arange(start=1, end=number_of_logits + 1, device=device,step=1, dtype=input.dtype).view(1, -1)
        range = range.expand_as(zs)

        bound = 1 + range * zs
        cumulative_sum_zs = torch.cumsum(zs, dim)
        is_gt = torch.gt(bound, cumulative_sum_zs).type(input.type())
        k = torch.max(is_gt * range, dim, keepdim=True)[0]
        zs_sparse = is_gt * zs
        taus = (torch.sum(zs_sparse, dim, keepdim=True) - 1) / k
        taus = taus.expand_as(input)
        self.output = torch.max(torch.zeros_like(input), input - taus)
        output = self.output
        output = output.transpose(0, 1)
        output = output.reshape(original_size)
        output = output.transpose(0, self.dim)
        return output
    def backward(self, grad_output):
        dim = 1
        nonzeros = torch.ne(self.output, 0)
        sum = torch.sum(grad_output * nonzeros, dim=dim) / torch.sum(nonzeros, dim=dim)
        self.grad_input = nonzeros * (grad_output - sum.expand_as(grad_output))
        return self.grad_input
    
def initialize_non_glu(module,inp_dim,out_dim):
    gain = np.sqrt((inp_dim+out_dim)/np.sqrt(4*inp_dim))
    torch.nn.init.xavier_normal_(module.weight, gain=gain)
    
class GBN(nn.Module):
    def __init__(self,inp,vbs=128,momentum=0.01):
        super().__init__()
        self.bn = nn.BatchNorm1d(inp,momentum=momentum)
        self.vbs = vbs
    def forward(self,x):
        chunk = torch.chunk(x,max(1,x.size(0)//self.vbs),0)
        res = [self.bn(y) for y in chunk ]
        return torch.cat(res,0)

class GLU(nn.Module):
    def __init__(self,inp_dim,out_dim,fc=None,vbs=128):
        super().__init__()
        if fc:
            self.fc = fc
        else:
            self.fc = nn.Linear(inp_dim,out_dim*2)
        self.bn = GBN(out_dim*2,vbs=vbs) 
        self.od = out_dim
    def forward(self,x):
        x = self.bn(self.fc(x))
        return x[:,:self.od]*torch.sigmoid(x[:,self.od:])
    

class FeatureTransformer(nn.Module):
    def __init__(self,inp_dim,out_dim,shared,n_ind,vbs=128):
        super().__init__()
        first = True
        self.shared = nn.ModuleList()
        if shared:
            self.shared.append(GLU(inp_dim,out_dim,shared[0],vbs=vbs))
            first= False    
            for fc in shared[1:]:
                self.shared.append(GLU(out_dim,out_dim,fc,vbs=vbs))
        else:
            self.shared = None
        self.independ = nn.ModuleList()
        if first:
            self.independ.append(GLU(inp,out_dim,vbs=vbs))
        for x in range(first, n_ind):
            self.independ.append(GLU(out_dim,out_dim,vbs=vbs))
        self.scale = torch.sqrt(torch.tensor([.5],device=device))
    def forward(self,x):
        if self.shared:
            x = self.shared[0](x)
            for glu in self.shared[1:]:
                x = torch.add(x, glu(x))
                x = x*self.scale
        for glu in self.independ:
            x = torch.add(x, glu(x))
            x = x*self.scale
        return x
    
class AttentionTransformer(nn.Module):
    def __init__(self,inp_dim,out_dim,relax,vbs=128):
        super().__init__()
        self.fc = nn.Linear(inp_dim,out_dim)
        self.bn = GBN(out_dim,vbs=vbs)
#         self.smax = Sparsemax()
        self.r = torch.tensor([relax],device=device)
    def forward(self,a,priors):
        a = self.bn(self.fc(a))
        mask = torch.sigmoid(a*priors)
        priors =priors*(self.r-mask)
        return mask

class DecisionStep(nn.Module):
    def __init__(self,inp_dim,n_d,n_a,shared,n_ind,relax,vbs=128):
        super().__init__()
        self.fea_tran = FeatureTransformer(inp_dim,n_d+n_a,shared,n_ind,vbs)
        self.atten_tran = AttentionTransformer(n_a,inp_dim,relax,vbs)
    def forward(self,x,a,priors):
        mask = self.atten_tran(a,priors)
        loss = ((-1)*mask*torch.log(mask+1e-10)).mean()
        x = self.fea_tran(x*mask)
        return x,loss

class TabNet(nn.Module):
    def __init__(self,inp_dim,final_out_dim,n_d=64,n_a=64,n_shared=2,n_ind=2,n_steps=5,relax=1.2,vbs=128):
        super().__init__()
        if n_shared>0:
            self.shared = nn.ModuleList()
            self.shared.append(nn.Linear(inp_dim,2*(n_d+n_a)))
            for x in range(n_shared-1):
                self.shared.append(nn.Linear(n_d+n_a,2*(n_d+n_a)))
        else:
            self.shared=None
        self.first_step = FeatureTransformer(inp_dim,n_d+n_a,self.shared,n_ind) 
        self.steps = nn.ModuleList()
        for x in range(n_steps-1):
            self.steps.append(DecisionStep(inp_dim,n_d,n_a,self.shared,n_ind,relax,vbs))
        self.fc = nn.Linear(n_d,final_out_dim)
        self.bn = nn.BatchNorm1d(inp_dim)
        self.n_d = n_d
    def forward(self,x):
        x = self.bn(x)
        x_a = self.first_step(x)[:,self.n_d:]
        loss = torch.zeros(1).to(x.device)
        out = torch.zeros(x.size(0),self.n_d).to(x.device)
        priors = torch.ones(x.shape).to(x.device)
        for step in self.steps:
            x_te,l = step(x,x_a,priors)
            out += F.relu(x_te[:,:self.n_d])
            x_a = x_te[:,self.n_d:]
            loss += l
        return self.fc(out),loss
    
class TabNetWithEmbed(nn.Module):
    def __init__(self,inp_dim,final_out_dim,n_d=64,n_a=64,n_shared=2,n_ind=2,n_steps=5,relax=1.2,vbs=128):
        super().__init__()
        self.tabnet = TabNet(inp_dim,final_out_dim,n_d,n_a,n_shared,n_ind,n_steps,relax,vbs)
        self.cat_embed = []
        self.emb1 = nn.Embedding(2,1)
        self.emb3 = nn.Embedding(3,1)
        self.cat_embed.append(self.emb1)
        self.cat_embed.append(self.emb3)
        
    def forward(self,catv,contv):
        catv = catv.to(device)
        contv = contv.to(device)
        embeddings = [embed(catv[:,idx]) for embed,idx in zip(self.cat_embed,range(catv.size(1)))]
        catv = torch.cat(embeddings,1)
        x = torch.cat((catv,contv),1).contiguous()
        x,l = self.tabnet(x)
        return torch.sigmoid(x),l

# Complete NN and thoughts

In [ ]:
# for this run of the nn I used a feature extraction backbone, then a VAE for dim-reduction, and then finally self attention and final output layer

# I would like to work more with DWT, but I don't know if it'll really be effective, I would have to try many configurations, find the one with the least error to complexity (number of coefficients).
# DWT/CWT coefficients may in fact be a good set of features for classification instead, I will have to read into the documentation of the library I'm using to extract them.  
# If I can avoid the DWT entirely by doing things end to end, that would be nice, however I will not discount feature augmentation, the melspectrogram kernels use some transforms which could be 
# useful here

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        #this convolutional upsample layer is a placeholder for if you are working with grayscale images and want to convert from (1,h,w) to (3,h,w)
        #self.conv_upsample = nn.ConvTranspose2d(in_channels = 1, out_channels=3,kernel_size=1)
        
        #I may use this in the future for denoising the signal or extracting signal coefficients 
        #self.decompose = DWT1DForward(wave='db6', J=3)

        #feature extracting backbone
        self.timm = timm.create_model('tf_efficientnetv2_b0',features_only =True)
        #VAE (dimensionality reduction)
        self.vae = VAE()
        #self attention, so as to create a better feature representation
        self.tabnet = TabNet(512,100)
        #Mixed Density Network (distribution modeling), this is likely unnecessary for the following problem because the problem has clearly been shown to be modelable
        #self.mdn = MDN(100,1,7)
        self.final = nn.Linear(100,1) 
    def forward(self,x):
        
        #x = self.conv_upsample(x)
        #print(x.shape)
        #x = torch.cat([torch.reshape(self.decompose(inp)[0],(1,3, 4096, 9)) for inp in x],dim=0) 
        
        # extract, flatten, and concatenate the feature maps of the input 
        tim = torch.cat([torch.reshape(output,(output.shape[0],-1)) for output in self.timm(x)],dim=1)
        #use the autencoder and the tabnet in one step
        out,loss = self.tabnet(self.vae(tim))
        #finally, output the results 
        return F.sigmoid(self.final(out))

# About the Competition🚩
<p style="font-size:15px">It's been said that teamwork makes the dream work. This couldn't be truer for the breakthrough discovery of gravitational waves (GW), signals from colliding binary black holes in 2015. It required the collaboration of experts in physics, mathematics, information science, and computing. GW signals have led researchers to observe a new population of massive, stellar-origin black holes, to unlock the mysteries of neutron star mergers, and to measure the expansion of the Universe. These signals are unimaginably tiny ripples in the fabric of space-time and even though the global network of GW detectors are some of the most sensitive instruments on the planet, the signals are buried in detector noise. Analysis of GW data and the detection of these signals is a crucial mission for the growing global network of increasingly sensitive GW detectors. These challenges in data analysis and noise characterization could be solved with the help of data science.
</p><p>
In this competition, you’ll aim to detect GW signals from the mergers of binary black holes. Specifically, you'll build a model to analyze simulated GW time-series data from a network of Earth-based detectors.
</p><p>
Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.
</p>

# Data Description

<div style="font-size:15px">
 We are given 2 npy files and 2 csv files:-
<ul>
    <li><code>train:</code> the training set files, one npy file per observation; labels are provided in a files shown below
</li>
    <li><code>test:</code> the test set files; you must predict the probability that the observation contains a gravitational wave</li>
    <li><code>training_labels.csv:</code> target values of whether the associated signal contains a gravitational wave</li>
    <li><code>sample_submission.csv:</code> a sample submission file in the correct format
</li>
</ul>    
</div>

# Train Dataset

# EDA

In [ ]:
train_labels = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
sample_submission = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

Lets take a look at train_labels

In [ ]:
train_labels.head()

In [ ]:
y = train_labels['target'].values

In [ ]:
path = list(train_labels['id'])
for i in range(len(path)):
    path[i] = '../input/g2net-gravitational-wave-detection/train/' +path[i][0]+'/'+path[i][1]+'/'+path[i][2]+'/' + path[i] + '.npy'

In [ ]:
device = torch.device("cuda")

In [ ]:
NNN = NN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(NNN.parameters())

In [ ]:
NNN.train()
batch_size = 10
n_epochs = 10
batch_iterations  = 17
for i in range(n_epochs):
    idxs = list(np.random.random_integers(0,len(path),size = batch_size))
    train_samples = torch.Tensor(np.array([np.load(pth) for pth in np.array(path)[idxs]]).reshape(-1,3,4096,1)).to(device)
    targets = torch.Tensor(y[idxs]).to(device)
    loss = 0
    for sub_step in range(batch_iterations):
        optimizer.zero_grad()
        outputs = NNN(train_samples)
        loss = criterion(outputs,targets.unsqueeze(1))
        loss.backward()
        print(loss.item())
        optimizer.step()
    torch.cuda.empty_cache()
    gc.collect() 

In [ ]:
torch.cuda.empty_cache()
gc.collect() 

In [ ]:
outputs

# Model

In [ ]:
def id_to_path(idx,train=True):
    path = '../input/g2net-gravitational-wave-detection/'
    if train:
        folder = 'train/'
    else:
        folder = 'test/'
    path+=folder+idx[0]+'/'+idx[1]+'/'+idx[2]+'/'+idx+'.npy'
    return path

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        signal = sample['signals']

        return {'signals': torch.from_numpy(signal)}

In [ ]:
class GravDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, paths , transform=ToTensor()):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.paths = paths
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        samp = np.load(self.paths[idx]).reshape(3,4096,1)
        
        sample = {'signals': samp}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
# train_idx =  train_labels['id'].values
test_idx = sample_submission['id'].values

In [ ]:
path = list(test_idx)
for i in range(len(path)):
    path[i] = '../input/g2net-gravitational-wave-detection/test/' +path[i][0]+'/'+path[i][1]+'/'+path[i][2]+'/' + path[i] + '.npy'

In [ ]:
dataset = GravDataset(path)
loader = DataLoader(dataset,batch_size=100,num_workers=4,shuffle = False)

Good rule of thumb, if you get much poorer results when evaluating your NN with float weights/parameters instead of double (high precision values), your NN may need additional complexity, or more training

In [ ]:
NNN = NNN.cuda().float()
clear_output()

In [ ]:
preds = []

In [ ]:
(.1*(len(path)/100))/(60**1 )

In [ ]:
NNN.eval()
with torch.no_grad():
    for batch in loader:
#         init_time = time.time()
        test_batch = batch['signals'].float().cuda()
        output = NNN(test_batch).cpu().numpy().ravel().tolist()
        preds.extend(output)
#         fin_time = time.time()
#         print(fin_time - init_time)
#     test_batch = torch.Tensor(np.array([np.load(pth) for pth in np.array(path)[idxs]]).reshape(-1,3,4096,1)).to(device)

In [ ]:
submission = pd.DataFrame({'id':sample_submission['id'],'target':preds})

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)

<h2><center>Work in Progress ... ⏳</center></h2>